In [ ]:
#Mount Google Drive with this Colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import sys
import re
import subprocess
import errno
import argparse
import csv
from scipy.ndimage import rotate
from astropy.io import fits
from numpy import linalg
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from urllib import request

def path_safe(path):
        original_umask = os.umask(0)
        try:
            os.makedirs(path, mode=0o775)
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise
        os.umask(original_umask)

def finder_cosmos_acs_py3(data):
    try:

        id = str(data["ObjNo"])
        ra = data["RA"]
        dec = data["DEC"]
        resize_factor = 1+np.tan(np.deg2rad(7.76))/(1+np.tan(np.deg2rad(7.76)))
        size = 3*data["rKron1"]/60*resize_factor

        out_sub_dir = "fits"

        if ra < 0.0 or ra > 360.0:
            print('RA outside range 0.0-360.0:  ',ra, ' Quit.')
            exit()
        if dec < -90.0 or dec > 90.0:
            print('Dec outside range -90.0-90.0:  ',dec, ' Quit')
            exit()
        if size < 0.05 or size > 60.0:
            print('Size outside range 0.05-60.0:  ',size, 'Quit.')
            exit()

        outdir = os.getcwd() + '/' + out_sub_dir
        path_safe(outdir)

        #Correction values because not all of the base image contains information
        ra_corr = np.array([-0.0007412,0.0025433,0.0008265,-0.0013785])
        dec_corr = np.array([-0.0019341,-0.0055036,0.0019889,0.0039687])

        f_subsets=['I']
        for f in f_subsets:
            irsa_call = "https://irsa.ipac.caltech.edu/IBE?table=cosmos.cosmos_acs_2_0&POS=" + str(ra) + ',' + str(dec) + "&ct=csv&where=file_type=science"
            print(irsa_call)
            filename = outdir + '/cosmos_acs_2_0_' + id+ "_"+str(f) + '.csv'
            request.urlretrieve(irsa_call, filename)

            try:
                with open(filename) as csvfile:
                    reader = csv.DictReader(csvfile)
                    file_downloaded = False
                    diffs = []
                    for line in reader:

                        #Galaxy coordinates
                        p_image = np.array([dec,ra])

                        #Vertices of the image
                        ras = np.array([float(line["ra1"]),float(line["ra2"]),float(line["ra3"]),float(line["ra4"])])
                        decs = np.array([float(line["dec1"]),float(line["dec2"]),float(line["dec3"]),float(line["dec4"])])

                        #Check if it is inverted
                        if float(line["ra1"]) < float(line["ra2"]):
                            ras_actual = ras - ra_corr
                            decs_actual = decs - dec_corr
                        else: #Inverted
                            ras_actual = ras - np.roll(ra_corr,2)
                            decs_actual = decs - np.roll(dec_corr,2)

                        vertices, vertices_actual = [], []
                        for i in range(len(ras)):
                            vertices.append([decs[i],ras[i]])
                            vertices_actual.append([decs_actual[i],ras_actual[i]])
                        #Calculate distances to all corrected image boundaries
                        distances = get_distances_to_axes(vertices,p_image)
                        distances_actual = get_distances_to_axes(vertices_actual,p_image)

                        #If the image point is closer to an outer axis than to an actual axis
                        #Then neglect that distance
                        for i in range(len(distances_actual)):
                            if distances[i] < distances_actual[i]: distances_actual[i] = 0

                        #Select the minimum distance
                        diffs.append(np.min(distances_actual))


            except Exception as e:
                print(e)

            best_row = np.argmax(diffs)
            with open(filename) as csvfile:
                reader = csv.DictReader(csvfile)
                i = 0
                for line in reader:
                    if i == best_row:
                        fname = line['fname']
                        irsa_url = 'https://irsa.ipac.caltech.edu/ibe/data/cosmos/cosmos_acs_2_0/'+fname+'?center='+str(ra)+','+str(dec)+'&size='+str(size)+'arcmin'

                        request.urlretrieve(irsa_url,filename)
                        break
                    i = i + 1

            filelist = os.listdir(outdir)
            for file in filelist:
                if (file.endswith('.gz')):
                    filefull = os.path.join(outdir, file)
                    if (re.search(r':.* text',subprocess.Popen(["file",filefull],stdout=subprocess.PIPE).stdout.read()) is not None):
                        print('No image found, deleting: ', filefull)
                        os.remove(filefull)

        #Rotate image according to the WCS
        image_data = fits.getdata("fits/cosmos_acs_"+id+".fits", ext=0)
        header_data = fits.getheader("fits/cosmos_acs_"+id+".fits",ext=0)
        angle = float(header_data["ORIENTAT"])

        #Rotate
        im = Image.fromarray(image_data)
        image_data_rot = np.asarray(im.rotate(-angle,expand=1,resample=Image.BICUBIC))

        im = Image.fromarray(image_data_rot)
        width, height = im.size   # Get dimensions

        #Crop parameters
        alpha = np.deg2rad(angle-90)
        new_width = np.tan(alpha)**2/((1+np.tan(alpha))**2*np.sin(alpha)**2)*width
        new_height = np.tan(alpha)**2/((1+np.tan(alpha))**2*np.sin(alpha)**2)*height

        left = (width - new_width)/2
        top = (height - new_height)/2
        right = (width + new_width)/2
        bottom = (height + new_height)/2

        # Crop the center of the image
        im_result = np.asarray(im.crop((left, top, right, bottom)))

        header_data["ORIENTAT"] = 0
        header_data["PA_APER"] = 0
        hdu = fits.PrimaryHDU(im_result,header_data)
        hdu.writeto('fits/cosmos_acs_'+id+'.fits',overwrite=True)

    except Exception as e:
        print(e)

def calc_distance(p1,p2,p_image):
    return np.cross(p2-p1, p_image-p1)/linalg.norm(p2-p1)

def get_distances_to_axes(vertices,p_image):
    distances = []
    vertices.append(vertices[0])
    for i in range(len(vertices)-1):
        distances.append(calc_distance(np.array(vertices[i+1]),np.array(vertices[i]),p_image))
    return distances

In [ ]:
import glob, os
import numpy as np
import pandas as pd
import time
from multiprocessing.pool import ThreadPool as Pool
from multiprocessing.pool import ThreadPool
import multiprocessing as mp
from functools import partial
from PIL import Image
from tqdm import tqdm

data = pd.read_csv('/content/drive/MyDrive/MPE/ZoobotEuclid/Data/Hubble_COSMOS_labels_complete.csv')

starttime = time.time()
#Convert to list of dicts
data_list = []
print("Preparing data...")
for i in tqdm(range(len(data))):
    data_row = data.iloc[i]
    data_dict = data_row.to_dict()
    data_list.append(data_dict)


print("Downloading data...")
pool = Pool(128)
list(tqdm(pool.imap(finder_cosmos_acs_py3, data_list)))
print('\n That took {} seconds'.format(time.time() - starttime))

In [ ]:
!zip -r /content/fits_hubble_complete.zip /content/fits

In [ ]:
!cp fits_hubble_complete.zip /content/drive/MyDrive/MPE/ZoobotEuclid/Data/Hubble_COSMOS_complete/

In [ ]:
fits_files = glob.glob("/content/fits/*.fits")
print(len(fits_files))